<a href="https://colab.research.google.com/github/loicbi/Databricks/blob/develop/08_Databricks_%7C_Pyspark_Add%2C_Rename_and_Drop_Columns.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install findspark

In [3]:
!pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.0/317.0 MB 3.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.1-py2.py3-none-any.whl size=317488491 sha256=0b0d945782e64c698e922df92d40e74c83b4a31cdd4ec5fc888352032ddb8acd
  Stored in directory: /root/.cache/pip/wheels/80/1d/60/2c256ed38dddce2fdd93be545214a63e02fbd8d74fb0b7f3a6
Successfully built pyspark


In [4]:
import findspark
from pyspark.sql.functions import to_date, col, split, char_length, lit, trim, concat, to_char
# findspark.init()
from pyspark.sql import SparkSession
# Create a SparkSession
spark = SparkSession.builder.getOrCreate()
# spark = SparkSession.builder.appName("app_name").getOrCreate()
spark.conf.set("spark.sql.legacy.timeParserPolicy", "LEGACY")
df = spark.read.format('csv').option('header', True)\
          .option('inferSchema', True)\
          .option('sep', ',')\
          .load('/content/drive/MyDrive/DATA_ENGINEER/DATABRICKS/AAAA_________IAM___CODING_____Colab Notebooks/Learning/datasset/AFCON/Africa Cup of Nations Matches.csv')

df.printSchema()
df.show(10)


root
 |-- Year: integer (nullable = true)
 |-- Date : string (nullable = true)
 |-- Time : string (nullable = true)
 |-- HomeTeam: string (nullable = true)
 |-- AwayTeam: string (nullable = true)
 |-- HomeTeamGoals: double (nullable = true)
 |-- AwayTeamGoals: double (nullable = true)
 |-- Stage: string (nullable = true)
 |-- SpecialWinConditions: string (nullable = true)
 |-- Stadium: string (nullable = true)
 |-- City: string (nullable = true)
 |-- Attendance: double (nullable = true)

+----+---------+-----+---------+-------------+-------------+-------------+-----------------+--------------------+--------------------+------------+----------+
|Year|    Date |Time | HomeTeam|     AwayTeam|HomeTeamGoals|AwayTeamGoals|            Stage|SpecialWinConditions|             Stadium|        City|Attendance|
+----+---------+-----+---------+-------------+-------------+-------------+-----------------+--------------------+--------------------+------------+----------+
|1957|10-Feb-57| NULL|   Sudan

## Remove space in columns and rows

In [11]:
df_result = df.withColumn('Year_2', trim(df.Year + 1)).show()

+----+---------+-----+---------+-------------+-------------+-------------+-----------------+--------------------+--------------------+------------+----------+------+
|Year|    Date |Time | HomeTeam|     AwayTeam|HomeTeamGoals|AwayTeamGoals|            Stage|SpecialWinConditions|             Stadium|        City|Attendance|Year_2|
+----+---------+-----+---------+-------------+-------------+-------------+-----------------+--------------------+--------------------+------------+----------+------+
|1957|10-Feb-57| NULL|   Sudan |        Egypt|          1.0|          2.0|       Semifinals|                NULL|   Municipal Stadium|    Khartoum|   30000.0|  1958|
|1957|10-Feb-57| NULL|Ethiopia | South Africa|         NULL|         NULL|       Semifinals|Ethiopia  wins du...|                NULL|        NULL|      NULL|  1958|
|1957|16-Feb-57| NULL|   Egypt |     Ethiopia|          4.0|          0.0|            Final|                NULL|   Municipal Stadium|    Khartoum|   30000.0|  1958|
|195

In [ ]:
# Create a list of tuples where the first element is the old column name and the second element is the new column name
rename_columns = [(c, c.strip()) for c in df.columns]

# Use the withColumnRenamed method with a dictionary to rename multiple columns

for current_name, new_name in rename_columns:
    df = df.withColumnRenamed(current_name, new_name)
    df = df.withColumn(new_name, trim(df[new_name]))


# Print the schema of the renamed DataFrame
df.printSchema()
df.show(10)

root
 |-- Year: string (nullable = true)
 |-- Date: string (nullable = true)
 |-- Time: string (nullable = true)
 |-- HomeTeam: string (nullable = true)
 |-- AwayTeam: string (nullable = true)
 |-- HomeTeamGoals: string (nullable = true)
 |-- AwayTeamGoals: string (nullable = true)
 |-- Stage: string (nullable = true)
 |-- SpecialWinConditions: string (nullable = true)
 |-- Stadium: string (nullable = true)
 |-- City: string (nullable = true)
 |-- Attendance: string (nullable = true)

+----+---------+----+--------+------------+-------------+-------------+-----------------+--------------------+--------------------+-----------+----------+
|Year|     Date|Time|HomeTeam|    AwayTeam|HomeTeamGoals|AwayTeamGoals|            Stage|SpecialWinConditions|             Stadium|       City|Attendance|
+----+---------+----+--------+------------+-------------+-------------+-----------------+--------------------+--------------------+-----------+----------+
|1957|10-Feb-57|NULL|   Sudan|       Egypt|  

# Add column ===> withColumn()  or withColumns()
## lit() : constant literal


In [ ]:
# add new column by calculation
df_add_column = df.withColumn('DateCast_to_YYYY-MM-DD', lit(to_date(df['Date'], 'dd-MMM-yy')))

# add multiple columns
df_add_column = df_add_column.withColumns({'homeTeam # AwayTeam': concat(df_add_column['homeTeam'],  lit(' ') , df_add_column['AwayTeam']), 'ColumnAdded': lit(999999999999999999)})
# + lit(' ') + df_add_column['AwayTeam']
# df_add_column.printSchema()
df_add_column.show(2)

+----+---------+----+--------+------------+-------------+-------------+----------+--------------------+-----------------+--------+----------+----------------------+--------------------+------------------+
|Year|     Date|Time|HomeTeam|    AwayTeam|HomeTeamGoals|AwayTeamGoals|     Stage|SpecialWinConditions|          Stadium|    City|Attendance|DateCast_to_YYYY-MM-DD| homeTeam # AwayTeam|       ColumnAdded|
+----+---------+----+--------+------------+-------------+-------------+----------+--------------------+-----------------+--------+----------+----------------------+--------------------+------------------+
|1957|10-Feb-57|NULL|   Sudan|       Egypt|          1.0|          2.0|Semifinals|                NULL|Municipal Stadium|Khartoum|   30000.0|            1957-02-10|         Sudan Egypt|999999999999999999|
|1957|10-Feb-57|NULL|Ethiopia|South Africa|         NULL|         NULL|Semifinals|Ethiopia  wins du...|             NULL|    NULL|      NULL|            1957-02-10|Ethiopia South A

# Rename column ===> withColumnRenamed() or withColumnsRenamed

In [ ]:
df_rename_column = df_add_column.withColumnsRenamed({'DateCast_to_YYYY-MM-DD': 'DateCast', 'Attendance': '_renamed__Attendance'})
display(df_rename_column)

DataFrame[Year: string, Date: string, Time: string, HomeTeam: string, AwayTeam: string, HomeTeamGoals: string, AwayTeamGoals: string, Stage: string, SpecialWinConditions: string, Stadium: string, City: string, _renamed__Attendance: string, DateCast: date, homeTeam # AwayTeam: string]

# Remove column ===> drop()

In [ ]:
df_remove_col = df_rename_column.drop('_renamed__Attendance')
df_remove_col.printSchema()

root
 |-- Year: string (nullable = true)
 |-- Date: string (nullable = true)
 |-- Time: string (nullable = true)
 |-- HomeTeam: string (nullable = true)
 |-- AwayTeam: string (nullable = true)
 |-- HomeTeamGoals: string (nullable = true)
 |-- AwayTeamGoals: string (nullable = true)
 |-- Stage: string (nullable = true)
 |-- SpecialWinConditions: string (nullable = true)
 |-- Stadium: string (nullable = true)
 |-- City: string (nullable = true)
 |-- DateCast: date (nullable = true)
 |-- homeTeam # AwayTeam: string (nullable = true)

